In [ ]:
!pip install fastparquet &> /dev/null
!pip install pyarrow &> /dev/null
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#Get all folder names in S3 bucket
import boto3
s3 = boto3.resource('s3')
my_bucket = s3.Bucket('atharva-54')

In [ ]:
def get_folder_name(path):
    return path.split('/*.csv')

# files = [obj.key for obj in my_bucket.objects.all()]
parquet_files = [obj.key for obj in my_bucket.objects.filter(Prefix='PK')]
parquet_files

In [ ]:
for obj in my_bucket.objects.all():
    print(obj.key, obj.size)

In [ ]:
#Read xlsx file AttributeError: 'ElementTree' object has no attribute 'getiterator'
theme_list = pd.read_excel('GKG theme list for R54.xlsx', engine="openpyxl")
#Remove rows with NaN values in all columns except 'GKG theme' and 'Human-readable'
theme_list = theme_list.dropna(how='all', subset=['th_topic_2a_environment',
       'th_topic_2b_regulation', 'topic_2a_2b_combined',
       'th_topic_4_customer_orientation', 'th_topic_6_shareholder_affiliation',
       'th_topic_8_hiring_employees', 'th_topic_9_natural_disaster',
       'th_topic_11_litigation', 'th_topic_12_carbon_emissions',
       'th_topic_13_labor_issues', 'topic_8_13_combined',
       'th_topic_14_supply_chain_problems', 'th_topic_15_taxes']).reset_index(drop=True)
#Drop Human-readable column
theme_list = theme_list.drop(columns=['Human-readable'])
#Make GKG theme uppercase
theme_list['GKG theme'] = theme_list['GKG theme'].str.upper()
#Drop duplicates in GKG theme column
theme_list = theme_list.drop_duplicates(subset=['GKG theme'], keep='first').reset_index(drop=True)
theme_list

In [ ]:
#If th_topic_2a_environment & th_topic_2b_regulation are both 1, then topic_2a_2b combined = 1
theme_list['topic_2a_2b_combined'] = np.where((theme_list['th_topic_2a_environment'] == 1) & (theme_list['th_topic_2b_regulation'] == 1), 1, 0)

#If th_topic_8_hiring_employees OR th_topic_13_labor_issues are both 1, then topic_8_13_combined = 1
theme_list['topic_8_13_combined'] = np.where((theme_list['th_topic_8_hiring_employees'] == 1) | (theme_list['th_topic_13_labor_issues'] == 1), 1, 0)

In [ ]:
#main
main_columns = ['th_topic_2a_environment', 'th_topic_2b_regulation',
       'topic_2a_2b_combined', 'th_topic_4_customer_orientation',
       'th_topic_6_shareholder_affiliation', 'th_topic_8_hiring_employees',
       'th_topic_9_natural_disaster', 'th_topic_11_litigation',
       'th_topic_12_carbon_emissions', 'th_topic_13_labor_issues',
       'topic_8_13_combined', 'th_topic_14_supply_chain_problems',
       'th_topic_15_taxes']

In [ ]:
imp_themes = list(theme_list['GKG theme'])
#Capitalize all themes
imp_themes = [x.upper() for x in imp_themes]

In [ ]:
#Create a list of themes for each main column
for i in main_columns:
    globals()[i + '_themes'] = list(theme_list[theme_list[i] > 0]['GKG theme'])

In [ ]:
# th_topic_2a_environment_themes is a list. Add "mean_" to each element in the list
th_topic_2a_environment_themes_mean = ['mean_' + i for i in th_topic_2a_environment_themes]
th_topic_2b_regulation_themes_mean = ['mean_' + i for i in th_topic_2b_regulation_themes]
topic_2a_2b_combined_themes_mean = ['mean_' + i for i in topic_2a_2b_combined_themes]
th_topic_4_customer_orientation_themes_mean = ['mean_' + i for i in th_topic_4_customer_orientation_themes]
th_topic_6_shareholder_affiliation_themes_mean = ['mean_' + i for i in th_topic_6_shareholder_affiliation_themes]
th_topic_8_hiring_employees_themes_mean = ['mean_' + i for i in th_topic_8_hiring_employees_themes]
th_topic_9_natural_disaster_themes_mean = ['mean_' + i for i in th_topic_9_natural_disaster_themes]
th_topic_11_litigation_themes_mean = ['mean_' + i for i in th_topic_11_litigation_themes]
th_topic_12_carbon_emissions_themes_mean = ['mean_' + i for i in th_topic_12_carbon_emissions_themes]
th_topic_13_labor_issues_themes_mean = ['mean_' + i for i in th_topic_13_labor_issues_themes]
topic_8_13_combined_themes_mean = ['mean_' + i for i in topic_8_13_combined_themes]
th_topic_14_supply_chain_problems_themes_mean = ['mean_' + i for i in th_topic_14_supply_chain_problems_themes]
th_topic_15_taxes_themes_mean = ['mean_' + i for i in th_topic_15_taxes_themes]

#For count
th_topic_2a_environment_themes_count = ['count_' + i for i in th_topic_2a_environment_themes]
th_topic_2b_regulation_themes_count = ['count_' + i for i in th_topic_2b_regulation_themes]
topic_2a_2b_combined_themes_count = ['count_' + i for i in topic_2a_2b_combined_themes]
th_topic_4_customer_orientation_themes_count = ['count_' + i for i in th_topic_4_customer_orientation_themes]
th_topic_6_shareholder_affiliation_themes_count = ['count_' + i for i in th_topic_6_shareholder_affiliation_themes]
th_topic_8_hiring_employees_themes_count = ['count_' + i for i in th_topic_8_hiring_employees_themes]
th_topic_9_natural_disaster_themes_count = ['count_' + i for i in th_topic_9_natural_disaster_themes]
th_topic_11_litigation_themes_count = ['count_' + i for i in th_topic_11_litigation_themes]
th_topic_12_carbon_emissions_themes_count = ['count_' + i for i in th_topic_12_carbon_emissions_themes]
th_topic_13_labor_issues_themes_count = ['count_' + i for i in th_topic_13_labor_issues_themes]
topic_8_13_combined_themes_count = ['count_' + i for i in topic_8_13_combined_themes]
th_topic_14_supply_chain_problems_themes_count = ['count_' + i for i in th_topic_14_supply_chain_problems_themes]
th_topic_15_taxes_themes_count = ['count_' + i for i in th_topic_15_taxes_themes]

In [ ]:
#Read PRIO GRID spine
spine = pd.read_csv('PRIO GRID spine.csv')

def lat_long_to_PG_lat_long(x):
    list_lat_long = []
    for i in range(len(x)):
        if x[i] >= 0:
            pg_val = int(x[i]) + 0.25 if x[i] % 1 >= 0 and x[i] % 1 <= 0.5 else int(x[i]) + 0.75
        else:
            pg_val = int(x[i]) - 0.75 if x[i] % 1 >= 0 and x[i] % 1 <= 0.5 else int(x[i]) - 0.25
        list_lat_long.append(pg_val)
    return list_lat_long

In [ ]:
countries = ['PK', 'CO', 'WI', 'NS', 'LI', 'CA', 'NC', 'WE', 'BG', 'GR', 'FI', 'QA', 'VM', 'LY', 'BR', 'MY', 'EU', 'BU', 'FK', 'RO', 'KN', 'AE', 'RS', 'BA', 'VI', 'UG', 'PO', 'TK', 'BL', 'RE', 'DA', 'EI', 'AU', 'TN', 'KT', 'MG', 'BC', 'MA', 'CG', 'AY', 'SH', 'SP', 'NF', 'AS', 'VC', 'GL', 'RN', 'JN', 'FJ', 'MD', 'UP', 'BO', 'IN', 'HK', 'MV', 'GI', 'MU', 'MP', 'GP', 'GT', 'SZ', 'LS', 'AO', 'RM', 'TH', 'JM', 'FP', 'MT', 'PC', 'PP', 'SM', 'AJ', 'LT', 'EZ', 'KU', 'AC', 'RI', 'CH', 'BK', 'CW', 'EG', 'DJ', 'GM', 'OS', 'TU', 'CT', 'BN', 'PF', 'NH', 'MZ', 'ID', 'KZ', 'FO', 'BB', 'SV', 'CU', 'OC', 'NG', 'AQ', 'ET', 'TW', 'LO', 'GH', 'IM', 'GK', 'FG', 'LQ', 'CJ', 'NP', 'CI', 'TL', 'MN', 'GZ', 'HR', 'SU', 'HU', 'BD', 'LG', 'CY', 'CN', 'OD', 'PE', 'UV', 'SC', 'GJ', 'MK', 'GO', 'BV', 'CQ', 'BP', 'TE', 'EK', 'AR', 'ST', 'IC', 'SE', 'DR', 'BF', 'IS', 'TI', 'SL', 'MI', 'PG', 'UK', 'NZ', 'WA', 'MC', 'HO', 'LE', 'IO', 'DQ', 'AM', 'CD', 'NE', 'UY', 'JE', 'ER', 'BE', 'KR', 'WS', 'SY', 'IZ', 'CS', 'AG', 'NL', 'ML', 'GQ', 'CB', 'SG', 'IP', 'MB', 'IT', 'TX', 'LU', 'PA', 'LH', 'RP', 'AF', 'NT', 'PU', 'CF', 'WZ', 'EC', 'NI', 'TV', 'BY', 'NU', 'FR', 'NO', 'BX', 'NR', 'IV', 'TP', 'MH', 'KS', 'BT', 'JO', 'AV', 'TD', 'TZ', 'JA', 'KV', 'SA', 'YI', 'SF', 'GV', 'YM', 'SO', 'HQ', 'HA', 'JQ', 'TS', 'VT', 'SW', 'ZI', 'SB', 'VE', 'TT', 'GB', 'KE', 'GG', 'FM', 'CV', 'CM', 'KQ', 'MO', 'SN', 'ZA', 'TO', 'WQ', 'PL', 'CE', 'RW', 'KG', 'SI', 'MR', 'MX', 'EN', 'ES', 'RB', 'MF', 'GA', 'LA', 'AL', 'US', 'MQ', 'BQ', 'UZ', 'PM', 'FQ', 'MJ', 'DO', 'GY', 'BH', 'IR', 'BM', 'PS', 'VQ', 'AN', 'RQ', 'CR', 'JU', 'WF', 'CK', 'UC', 'FS', 'HM', 'TB', 'NN', 'SX']

In [ ]:
#Gather all split data into one dataframe

def gather(countries):
    #Make an empty dataframe to store the appended data
    for country in countries:
        parquet_files = [obj.key for obj in my_bucket.objects.filter(Prefix=country)]
        print(country)
    
        df_main = pd.DataFrame()
        for filer in parquet_files:
            print(filer)
            #Read parquet file and specify columns to read
            df = pd.read_parquet('s3://atharva-54/'+filer, columns=['date', 'tone', 'themes', 'locations'])

            #Keep date from timestamp
            df['date'] = df['date'].dt.date

            #Keep "tone" from "tone" column which is a dictionary
            df['tone'] = df['tone'].apply(lambda x: x['tone'])

            #Get "theme" from each dictionary and make a list of themes
            df['themes'] = df['themes'].apply(lambda x: [i['theme'] for i in x] if x is not None else None)

            #Explode themes column
            df = df.explode('themes').reset_index(drop=True)

            #Keep only rows with themes in imp_themes
            df = df[df['themes'].isin(imp_themes)].reset_index(drop=True)

            #Get "location" from each dictionary and make a list of locations. If there are duplicates in the list, keep only one
            df['locations'] = df['locations'].apply(lambda x: [[i['location_latitude'], i['location_longitude']] for i in x] if x is not None else None)

            #Explode locations column
            df = df.explode('locations').reset_index(drop=True)

            #From location, keep first value as latitude and second value as longitude
            df['latitude'] = df['locations'].apply(lambda x: x[0])
            df['longitude'] = df['locations'].apply(lambda x: x[1])

            #Drop locations column
            df = df.drop(columns=['locations'])

            #Remove duplicate rows
            df = df.drop_duplicates().reset_index(drop=True)

            #Create new columns for PG latitude and longitude
            df['lat'] = np.nan
            df['lon'] = np.nan
            
            #drop row with latitude column = ''
            df = df[df['latitude'] != ''].reset_index(drop=True)
            df = df[df['longitude'] != ''].reset_index(drop=True)

            #Make latitude and longitude columns into float
            df['latitude'] = df['latitude'].astype(float)
            df['longitude'] = df['longitude'].astype(float)

            #Convert latitude and longitude to PG latitude and longitude
            df['lat'] = lat_long_to_PG_lat_long(df["latitude"])
            df['lon'] = lat_long_to_PG_lat_long(df["longitude"])

            #Drop latitude and longitude columns
            df = df.drop(columns=['latitude', 'longitude'])

            #Merge df with spine
            df = df.merge(spine, how='left', left_on=['lat', 'lon'], right_on=['lat', 'lon']).reset_index(drop=True)

            #Drop lat and lon columns
            df = df.drop(columns=['lat', 'lon'])

            #Drop rows with NaN values in gid column
            df = df.dropna(subset=['gid']).reset_index(drop=True)

            #Make GID column into int
            df['gid'] = df['gid'].astype(int)

            #Convert tone column to float
            df['tone'] = df['tone'].astype(float)

            #Convert date column to datetime
            df['date'] = pd.to_datetime(df['date'])

            # Create a new DataFrame to store the appended themes
            new_df = pd.DataFrame({'themes': imp_themes})
            # Fill the date column with the previous value
            new_df['date'] = df['date'].ffill()[0]
            # Fill the tone and gid columns with zeros
            new_df['tone'] = 0
            new_df['gid'] = 0

            # Append the new DataFrame to the original DataFrame
            df = pd.concat([df, new_df], ignore_index=True)

            # Fill all remaining NaN values with np.NaN
            df = df.fillna(np.NaN)

            #Less Efficient: Append imp_themes to df under column 'themes' and fill other columns with np.NaN
            # for i in imp_themes:
            #     df = df.append({'themes':i}, ignore_index=True)
            #     #Date column to be ffilled
            #     df['date'] = df['date'].ffill()
            #     #Fill tone & gid column with 0
            #     df['tone'] = df['tone'].fillna(0)
            #     df['gid'] = df['gid'].fillna(0)
            # df = df.fillna(np.NaN)

            #Pivot table to get mean tone and count of themes per gid and date
            df = df.pivot_table(index=['gid', 'date'], columns='themes', values='tone', aggfunc=['mean', 'count']).reset_index().sort_values(by=['gid', 'date'])

            #make only one level of column names, add upper level name to lower level name
            df.columns = [' '.join(col).strip() for col in df.columns.values]

            #Drop gid == 0
            df = df[df['gid'] != 0].reset_index(drop=True)

            #Sort by gid
            df = df.sort_values(by=['gid']).reset_index(drop=True)

            #Replace whitespace in column names with underscore
            df.columns = df.columns.str.replace(' ', '_')   

            #Take column wise mean of column names present in th_topic_2a_environment_themes for each row and store in column "average_th_topic_2a_environment"
            df['average_th_topic_2a_environment'] = df[th_topic_2a_environment_themes_mean].mean(axis=1)
            df['average_th_topic_2b_regulation'] = df[th_topic_2b_regulation_themes_mean].mean(axis=1)
            df['average_topic_2a_2b_combined'] = df[topic_2a_2b_combined_themes_mean].mean(axis=1)
            df['average_th_topic_4_customer_orientation'] = df[th_topic_4_customer_orientation_themes_mean].mean(axis=1)
            df['average_th_topic_6_shareholder_affiliation'] = df[th_topic_6_shareholder_affiliation_themes_mean].mean(axis=1)
            df['average_th_topic_8_hiring_employees'] = df[th_topic_8_hiring_employees_themes_mean].mean(axis=1)
            df['average_th_topic_9_natural_disaster'] = df[th_topic_9_natural_disaster_themes_mean].mean(axis=1)
            df['average_th_topic_11_litigation'] = df[th_topic_11_litigation_themes_mean].mean(axis=1)
            df['average_th_topic_12_carbon_emissions'] = df[th_topic_12_carbon_emissions_themes_mean].mean(axis=1)
            df['average_th_topic_13_labor_issues'] = df[th_topic_13_labor_issues_themes_mean].mean(axis=1)
            df['average_topic_8_13_combined'] = df[topic_8_13_combined_themes_mean].mean(axis=1)
            df['average_th_topic_14_supply_chain_problems'] = df[th_topic_14_supply_chain_problems_themes_mean].mean(axis=1)
            df['average_th_topic_15_taxes'] = df[th_topic_15_taxes_themes_mean].mean(axis=1)

            #For count
            df['cnt_th_topic_2a_environment'] = df[th_topic_2a_environment_themes_count].sum(axis=1)
            df['cnt_th_topic_2b_regulation'] = df[th_topic_2b_regulation_themes_count].sum(axis=1)
            df['cnt_topic_2a_2b_combined'] = df[topic_2a_2b_combined_themes_count].sum(axis=1)
            df['cnt_th_topic_4_customer_orientation'] = df[th_topic_4_customer_orientation_themes_count].sum(axis=1)
            df['cnt_th_topic_6_shareholder_affiliation'] = df[th_topic_6_shareholder_affiliation_themes_count].sum(axis=1)
            df['cnt_th_topic_8_hiring_employees'] = df[th_topic_8_hiring_employees_themes_count].sum(axis=1)
            df['cnt_th_topic_9_natural_disaster'] = df[th_topic_9_natural_disaster_themes_count].sum(axis=1)
            df['cnt_th_topic_11_litigation'] = df[th_topic_11_litigation_themes_count].sum(axis=1)
            df['cnt_th_topic_12_carbon_emissions'] = df[th_topic_12_carbon_emissions_themes_count].sum(axis=1)
            df['cnt_th_topic_13_labor_issues'] = df[th_topic_13_labor_issues_themes_count].sum(axis=1)
            df['cnt_topic_8_13_combined'] = df[topic_8_13_combined_themes_count].sum(axis=1)
            df['cnt_th_topic_14_supply_chain_problems'] = df[th_topic_14_supply_chain_problems_themes_count].sum(axis=1)
            df['cnt_th_topic_15_taxes'] = df[th_topic_15_taxes_themes_count].sum(axis=1)

            #Delete all columns not beginning with 'average' or 'count'. Keep gid & date
            df = df[[col for col in df.columns if col.startswith('average') or col.startswith('cnt') or col.startswith('gid') or col.startswith('date')]]

            #Concatenate df_main with df
            df_main = pd.concat([df_main, df], ignore_index=True) 
        #Perform final groupby on df_main
        df_main_annual = pd.concat([df_main.groupby(['gid', df_main['date'].dt.year]).agg({col:'mean' for col in df_main.columns if col.startswith('average')}).reset_index(), df_main.groupby(['gid', df_main['date'].dt.year]).agg({col:'sum' for col in df_main.columns if col.startswith('cnt')}).reset_index(drop=True)], axis=1)
        df_main_quarterly = pd.concat([df_main.groupby(['gid', pd.PeriodIndex(df_main['date'], freq='Q')]).agg({col:'mean' for col in df_main.columns if col.startswith('average')}).reset_index(), df_main.groupby(['gid', pd.PeriodIndex(df_main['date'], freq='Q')]).agg({col:'sum' for col in df_main.columns if col.startswith('cnt')}).reset_index(drop=True)], axis=1)

        #take the mean of columns beginning with 'average', and store in column 'average_tone', row-wise
        df_main_annual['average_tone'] = df_main_annual[[col for col in df_main_annual.columns if col.startswith('average')]].mean(axis=1)

        #take the sum of columns beginning with 'cnt', and store in column 'cnt_tone', row-wise
        df_main_annual['total_articles'] = df_main_annual[[col for col in df_main_annual.columns if col.startswith('cnt')]].sum(axis=1)

        #take the mean of columns beginning with 'average', and store in column 'average_tone', row-wise
        df_main_quarterly['average_tone'] = df_main_quarterly[[col for col in df_main_quarterly.columns if col.startswith('average')]].mean(axis=1)

        #take the sum of columns beginning with 'cnt', and store in column 'cnt_tone', row-wise
        df_main_quarterly['total_articles'] = df_main_quarterly[[col for col in df_main_quarterly.columns if col.startswith('cnt')]].sum(axis=1)

        return df_main_annual, df_main_quarterly

In [ ]:
gather(countries)